In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [4]:
exfoliator_url = []

In [5]:
for i in range (1,14):
    url = f'https://www.stylevana.com/en_EU/skincare/facial-cleanser/exfoliator-peeling-scrub.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    exfoliator_list = soup.find_all('div', class_='product-item-info')
    for item in exfoliator_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            exfoliator_url.append(link['href'])
print(len(exfoliator_url))

149


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

In [22]:
len(exfoliator_url)

0

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [11]:
exfoliator_url[157]

'https://www.stylevana.com/en_US/dearanchy-purifying-black-sugar-spa-peel-100ml.html'

In [6]:
exfoliators = []

In [7]:
for link in exfoliator_url[0:149]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name = soup.find('h1', class_='product-name-h1').text
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        price = (soup.find('span', class_='price').text.strip())
        best_seller = soup.find(class_="product-badge-icon").get('title')
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        details = 'no info'
        ingredients = 'no info'
        price = 'no info'
        best_seller = 'no'
        
    exfoliator_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients,
    'price': price,
    'best-seller': best_seller}
    
    exfoliators.append(exfoliator_info)

In [8]:
print(len(exfoliators))

149


## 3. PASARLO A DATAFRAME

In [9]:
exfoliator_df = pd.DataFrame(exfoliators)

In [11]:
exfoliators = exfoliator_df.to_csv('../data_/exfoliator.csv')

In [10]:
exfoliator_df

,name,details,ingredients,price,best-seller
0,Krave - Kale Lalu Yaha -200ml,no info,no info,no info,no
1,BEAUTY OF JOSEON - Apricot Peeling Gel - 120ml,Cruelty Free: Yes\n\n\nProduct Unit: 120ml\n\n...,[],4.79€,Cruelty Free
2,Isntree - Clear Skin AB Peeling Pads - 1pack (...,Cruelty Free: Yes\n\n\nProduct Unit: 70pc\n\n\...,[],11.69€,Cruelty Free
3,iUNIK - Lime Moisture Mild Peeling Gel - 120g,no info,no info,no info,no
4,G9 SKIN - Grapefruit Vita Peeling Gel,no info,no info,no info,no
...,...,...,...,...,...
144,Liberta - Tsubuporon Peeling Gel EX - 8g,no info,no info,no info,no
145,3W Clinic - Placenta Soft Peeling Gel -180ml,no info,no info,no info,no
146,It's Skin - Mangowhite Peeling Gel - 120ml,no info,no info,no info,no
147,Lirikos - Facial Eraser Mild Peeling Gel - 150ml,no info,no info,no info,no


In [12]:
exfoliator_df[exfoliator_df['price'] == 'no info']

,name,details,ingredients,price,best-seller
0,Krave - Kale Lalu Yaha -200ml,no info,no info,no info,no
3,iUNIK - Lime Moisture Mild Peeling Gel - 120g,no info,no info,no info,no
4,G9 SKIN - Grapefruit Vita Peeling Gel,no info,no info,no info,no
5,Secret Key -Lemon Sparkling Peeling Gel,no info,no info,no info,no
9,9wishes - Rice Powder Polish - 5g*1ea,no info,no info,no info,no
...,...,...,...,...,...
143,primera - Facial Mild Peeling - 150ml,no info,no info,no info,no
144,Liberta - Tsubuporon Peeling Gel EX - 8g,no info,no info,no info,no
145,3W Clinic - Placenta Soft Peeling Gel -180ml,no info,no info,no info,no
146,It's Skin - Mangowhite Peeling Gel - 120ml,no info,no info,no info,no
